<a href="https://colab.research.google.com/github/WMadaraChamudini/PC_Assignment3/blob/main/IT23292154_PC_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Matrix Multiplication-IT23292154-PC Assignment 03**

### **Serial Code**

In [ ]:
%%writefile serial_mat_mul.c
// compile: gcc -O2 -std=c11 serial_mat_mul.c -o serial_mat_mul
// run: ./serial_mat_mul N_A_row N_A_col N_B_row N_B_col

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

double* alloc_mat(int rows, int cols){
  double *m =(double*) malloc((size_t) rows*cols*sizeof(double));
  if (!m){
    perror("malloc");
    exit(1);
  }
  return m;
}

void init_ex(double *A, double *B, int N_A_row, int N_A_col, int N_B_row, int N_B_col){
  // Initialize Matrix A
  for (long i=0; i < (long)N_A_row*N_A_col; ++i){
    A[i]=(double)((i/N_A_col) + (i%N_A_col)+1);
  }
  // Initialize Matrix B
  for (long i=0; i < (long)N_B_row*N_B_col; ++i){
    B[i]=(double)( ((i/N_B_col)+1) * ((i % N_B_col)+2) );
  }
}

void print_mat(const char *name, double *M, int rows, int cols){
    printf("%s:\n", name);
    for (int i=0;i<rows;++i) {
        for (int j=0;j<cols;++j) {
            printf( "%8.2f ", M[(long)i*cols + j] );
        }
        printf("\n");
    }
}

int main(int argc, char **argv){
  int N_A_row=3, N_A_col=3, N_B_row=3, N_B_col=3;

  if (argc==5){
    N_A_row = atoi(argv[1]);
    N_A_col = atoi(argv[2]);
    N_B_row = atoi(argv[3]);
    N_B_col = atoi(argv[4]);
  } else {
    printf("Usage: %s N_A_row N_A_col N_B_row N_B_col\n", argv[0]);
    return 1;
  }

  if (N_A_row<=0 || N_A_col<=0 || N_B_row<=0 || N_B_col<=0){
    printf("Invalid dimensions. All dimensions must be positive.\n");
    return 1;
  }

  if (N_A_col != N_B_row){
    printf("Error: N_A_col must be equal to N_B_row for matrix multiplication.\n");
    return 1;
  }

  double *A = alloc_mat(N_A_row, N_A_col);
  double *B = alloc_mat(N_B_row, N_B_col);
  double *C = alloc_mat(N_A_row, N_B_col); // Result matrix C will have dimensions N_A_row x N_B_col

  init_ex(A,B, N_A_row, N_A_col, N_B_row, N_B_col);
  for (long i=0; i<(long)N_A_row*N_B_col; ++i){ // Initialize C based on its dimensions
    C[i]=0.0;
  }

  clock_t t0 = clock();

  for (int i=0;i<N_A_row;++i) {        // iterate over rows of A
    for (int j=0;j<N_B_col;++j) {      // iterate over columns of B
      double sum = 0.0;
      for (int k=0;k<N_A_col;++k) {    // iterate over columns of A (or rows of B)
        sum += A[(long)i*N_A_col + k] * B[(long)k*N_B_col + j];
      }
      C[(long)i*N_B_col + j] = sum; // Store result in C
    }
  }

  clock_t t1 = clock();

  //time for execution in seconds
  double Tot_t=(double) (t1-t0)/CLOCKS_PER_SEC;

  //print the matrix if N_A_row and N_B_col are smaller than 10
  if (N_A_row <= 10 && N_A_col <= 10 && N_B_row <= 10 && N_B_col <= 10){
    print_mat("Matrix A", A, N_A_row, N_A_col);
    printf("\n");
    print_mat("Matrix B", B, N_B_row, N_B_col);
    printf("\n");
    print_mat("Result Matrix C = A * B", C, N_A_row, N_B_col);
    printf("\n");
  }

  //checksum and per-row sums
  double checksum = 0.0;
  printf("Summary:\n");
  printf("Matrix A size: %d x %d\n", N_A_row, N_A_col);
  printf("Matrix B size: %d x %d\n", N_B_row, N_B_col);
  printf("Result Matrix C size: %d x %d\n", N_A_row, N_B_col);
  printf("Execution time: %.6f seconds\n", Tot_t);
  for (int i=0;i<N_A_row;++i){         //iterate over rows of C
    double rowSum = 0.0;
    for (int j=0;j<N_B_col;++j){       //iterate over columns of C
      rowSum += C[(long)i*N_B_col + j];
    }
    if (N_A_row <= 50 && N_B_col <= 50) {       // Only print row sums if result matrix dimensions < 50
      printf(" Row %2d sum = %.2f\n", i, rowSum);
    }
    checksum += rowSum;
  }
  printf("Checksum (sum of all elements) = %.6e\n", checksum);

  free(A);
  free(B);
  free(C);
  return 0;
}

In [ ]:
!gcc serial_mat_mul.c -o serial_mat_mul

In [ ]:
!./serial_mat_mul 4 3 3 5


In [ ]:
!./serial_mat_mul 40 4000 4000 40

In [ ]:
!./serial_mat_mul 4000 400 400 4000

### **OpenMP Code**

In [ ]:
%%writefile openmp_mat_mul.c
//Compile: gcc -O3 -fopenmp openmp_mat_mul.c -o openmp_mat_mul
//Run: ./openmp_mat_mul N_A_row N_A_col N_B_row N_B_col threads

#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

double* alloc_mat(int rows, int cols){
  double *m = (double*) malloc((size_t)rows*cols*sizeof(double));
  if (!m){
    perror("malloc");
    exit(1);
  }
  return m;
}

void init_ex(double *A, double *B, int N_A_row, int N_A_col, int N_B_row, int N_B_col){
  // Initialize Matrix A
  for (long i=0; i<(long)N_A_row*N_A_col; ++i){
    A[i] = (double) ((i/N_A_col)+(i%N_A_col)+1);
  }
  // Initialize Matrix B
  for (long i=0; i<(long)N_B_row*N_B_col; ++i){
    B[i] = (double)(((i/N_B_col)+1) * ((i%N_B_col)+2));
  }
}

void print_mat(const char *name, double *M, int rows, int cols){
  printf("%s:\n", name);
  for (int i=0; i<rows; ++i){
    for (int j=0;j<cols;++j){
      printf("%8.2f ", M [(long) i*cols+j]);
    }
    printf("\n");
  }
}

int main(int argc, char **argv){
  if (argc < 6){
    printf("Usage: %s N_A_row N_A_col N_B_row N_B_col threads\n", argv[0]);
    return 1;
  }
  int N_A_row = atoi(argv[1]);
  int N_A_col = atoi(argv[2]);
  int N_B_row = atoi(argv[3]);
  int N_B_col = atoi(argv[4]);
  int threads = atoi(argv[5]);

  if (N_A_row<=0 || N_A_col<=0 || N_B_row<=0 || N_B_col<=0 || threads<=0){
    printf("Invalid arguments. All dimensions and threads must be positive.\n");
    return 1;
  }

  if (N_A_col != N_B_row){
    printf("Error: N_A_col must be equal to N_B_row for matrix multiplication.\n");
    return 1;
  }

  omp_set_num_threads(threads);

  double *A = alloc_mat(N_A_row, N_A_col);
  double *B = alloc_mat(N_B_row, N_B_col);
  double *C = alloc_mat(N_A_row, N_B_col);  //matrix C will have dimensions N_A_row x N_B_col

  init_ex(A,B, N_A_row, N_A_col, N_B_row, N_B_col);
  for (long i=0; i<(long)N_A_row*N_B_col; ++i){  //Initialize C based on its dimensions
    C[i]=0.0;
  }

  double t0 = omp_get_wtime();

  #pragma omp parallel
  {
    #pragma omp for schedule(static)
    for (int i=0; i<N_A_row; ++i){          // iterate over rows of A
      for (int k=0;k<N_A_col;++k){          // iterate over columns of A (or rows of B)
        double a = A[(long)i*N_A_col + k];
        for (int j=0;j<N_B_col;++j) {       // iterate over columns of B
          C[(long)i*N_B_col + j] += a * B[(long)k*N_B_col + j];
        }
      }
    }
  }

  double t1 = omp_get_wtime();

  //time for execution in seconds
  double Tot_t = t1-t0;

  //print the matrix if N_A_row and N_B_col are smaller than 10
  if (N_A_row<=10 && N_A_col<=10 && N_B_row<=10 && N_B_col<=10){
    print_mat("Matrix A", A, N_A_row, N_A_col);
    printf("\n");
    print_mat("Matrix B", B, N_B_row, N_B_col);
    printf("\n");
    print_mat("Result Matrix C = A * B", C, N_A_row, N_B_col);
    printf("\n");
  }

  double checksum = 0.0;
  printf("Summary:\n");
  printf("Matrix A size: %d x %d\n", N_A_row, N_A_col);
  printf("Matrix B size: %d x %d\n", N_B_row, N_B_col);
  printf("Result Matrix C size: %d x %d\n", N_A_row, N_B_col);
  printf("Threads used: %d\n", threads);
  printf("Execution time: %.6f seconds\n", Tot_t);
  for (int i=0; i<N_A_row; ++i){ // Iterate over rows of C
    double rowSum=0.0;
    for (int j=0;j<N_B_col;++j){ // Iterate over columns of C
      rowSum += C[(long)i*N_B_col+j];
    }
    if (N_A_row <= 50 && N_B_col <= 50) {       // Only print row sums if result matrix dimensions < 50
      printf("Row %2d sum = %.2f\n", i,rowSum);
    }
    checksum+=rowSum;
  }
  printf("Checksum (sum of all elements) = %.6e\n", checksum);

  free(A);
  free(B);
  free(C);
  return 0;
}

In [ ]:
!gcc -O3 -fopenmp openmp_mat_mul.c -o openmp_mat_mul

In [ ]:
!./openmp_mat_mul 4 3 3 5 4

In [ ]:
!./openmp_mat_mul 40 4000 4000 40 2

In [ ]:
!./openmp_mat_mul 40 4000 4000 40 4

In [ ]:
!./openmp_mat_mul 4000 400 400 4000 1

In [ ]:
!./openmp_mat_mul 4000 400 400 4000 2

In [ ]:
!./openmp_mat_mul 4000 400 400 4000 4

**MPI Code**

In [ ]:
%%writefile mpi_mat_mul.c
// Compile: mpicc -O3 mpi_mat_mul.c -o mpi_mat_mul
// Run: mpirun --allow-run-as-root --oversubscribe -np P ./mpi_mat_mul N_A_row N_A_col N_B_row N_B_col <-- P is the number of processes

#include <stdio.h>
#include <stdlib.h>
#include <mpi.h>

double* alloc_mat(int rows, int cols){
  double *m = (double*) malloc((size_t)rows*cols*sizeof(double));
  if (!m){
    perror("malloc");
    exit(1);
  }
  return m;
}

void init_ex(double *A, double *B, int N_A_row, int N_A_col, int N_B_row, int N_B_col){
  // Initialize Matrix A
  for (long i=0; i<(long)N_A_row*N_A_col; i++){
    A[i] = (double)( (i/N_A_col)+(i%N_A_col)+1 );
  }
  // Initialize Matrix B
  for (long i=0; i<(long)N_B_row*N_B_col; i++){
    B[i] = (double)( ((i/N_B_col)+1) * ((i%N_B_col)+2) );
  }
}

void print_mat(const char *name, double *M, int rows, int cols){
  printf("%s:\n", name);
  for (int i=0; i<rows; i++){
    for (int j=0; j<cols; j++){
      printf("%8.2f ", M[(long)i*cols+j] );
    }
    printf("\n");
  }
}

int main(int argc, char **argv){
  MPI_Init(&argc, &argv);
  int rank, size;
  MPI_Comm_rank(MPI_COMM_WORLD, &rank);
  MPI_Comm_size(MPI_COMM_WORLD, &size);

  if (argc<5){
    if(rank==0){
      printf("Usage: %s N_A_row N_A_col N_B_row N_B_col\n", argv[0]);
    }
    MPI_Finalize();
    return 1;
  }

  int N_A_row = atoi(argv[1]);
  int N_A_col = atoi(argv[2]);
  int N_B_row = atoi(argv[3]);
  int N_B_col = atoi(argv[4]);

  if (N_A_row<=0 || N_A_col<=0 || N_B_row<=0 || N_B_col<=0){
    if(rank==0){
      printf("Invalid dimensions. All dimensions must be positive.\n");
    }
    MPI_Finalize();
    return 1;
  }

  if (N_A_col != N_B_row){
    if(rank==0){
      printf("Error: N_A_col must be equal to N_B_row for matrix multiplication.\n");
    }
    MPI_Finalize();
    return 1;
  }

  double *A = NULL, *B = NULL, *C = NULL;

  //allocate B on all processes, A and C only on root
  B = alloc_mat(N_B_row, N_B_col);

  if (rank==0){
    A = alloc_mat(N_A_row, N_A_col);
    C = alloc_mat(N_A_row, N_B_col);
    init_ex(A, B, N_A_row, N_A_col, N_B_row, N_B_col);
  }

  //Broadcast B to all processes
  MPI_Bcast(B, N_B_row * N_B_col, MPI_DOUBLE, 0, MPI_COMM_WORLD);

  //Row distribution determination for Matrix A
  int rows_per_proc = N_A_row/size;
  int extra = N_A_row%size;
  int start = rank*rows_per_proc+(rank<extra ? rank:extra);
  int end = start+rows_per_proc+(rank<extra ? 1:0);
  int num_rows = end-start;

  double *A_rows = alloc_mat(num_rows, N_A_col); //local rows of A
  double *C_rows = alloc_mat(num_rows, N_B_col); //local rows of C

  // Scatter rows of A from root to all processes
  // Using MPI_Send/MPI_Recv for custom scatter logic
  if(rank==0){
    for(int r=0; r<size; r++){
      int r_start = r*rows_per_proc+(r<extra ? r:extra);
      int r_end = r_start+rows_per_proc+(r<extra ? 1:0);
      int r_num = r_end-r_start;
      if(r==0){
        for(int i=0; i<r_num*N_A_col; i++){
          A_rows[i] = A[i];
        }
      }else{
        MPI_Send(A + (long)r_start*N_A_col, r_num*N_A_col, MPI_DOUBLE, r, 0, MPI_COMM_WORLD);
      }
    }
  }else{
    MPI_Recv(A_rows, num_rows*N_A_col, MPI_DOUBLE, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
  }

  //start time
  double t0 = MPI_Wtime();

  //compute local C_rows = A_rows*B
  for(int i=0; i<num_rows; i++){           //iterate over local rows of A_rows
    for(int j=0; j<N_B_col; j++){          //iterate over columns of B
      double sum = 0.0;
      for(int k=0; k<N_A_col; k++){        // iterate over columns of A_rows (or rows of B)
        sum += A_rows[(long)i*N_A_col+k] * B[(long)k*N_B_col+j];
      }
      C_rows[(long)i*N_B_col + j] = sum;
    }
  }

  //end time measurement before gathering results
  double t1 = MPI_Wtime();

  //total time
  double Tot_t = t1-t0;

  //gather results from all processes to root
  int *recvcounts = NULL;
  int *displs = NULL;
  if (rank==0){
    recvcounts = (int*) malloc(size * sizeof(int));
    displs = (int*) malloc(size * sizeof(int));
    int offset = 0;
    for(int r=0; r<size; r++){
      int r_start_global = r * rows_per_proc + (r<extra ? r:extra);
      int r_end_global = r_start_global + rows_per_proc + (r<extra ? 1:0);
      recvcounts[r] = (r_end_global - r_start_global) * N_B_col;  // No. of elements to receive from each process
      displs[r] = offset;      //Displacement for each process
      offset += recvcounts[r];
    }
  }

  MPI_Gatherv(C_rows, num_rows*N_B_col, MPI_DOUBLE, C, recvcounts, displs, MPI_DOUBLE, 0, MPI_COMM_WORLD);

  //reduce max execution time to root
  double max_time;
  MPI_Reduce(&Tot_t, &max_time, 1, MPI_DOUBLE, MPI_MAX, 0, MPI_COMM_WORLD);

  // Print results on root process
  if(rank==0){
    if(N_A_row<=10 && N_A_col<=10 && N_B_row<=10 && N_B_col<=10){
      print_mat("Matrix A", A, N_A_row, N_A_col);
      printf("\n");
      print_mat("Matrix B", B, N_B_row, N_B_col);
      printf("\n");
      print_mat("Result Matrix C = A*B", C, N_A_row, N_B_col);
      printf("\n");
    }

    double checksum=0.0;
    printf("Summary:\n");
    printf("Matrix A size: %d x %d\n", N_A_row,N_A_col);
    printf("Matrix B size: %d x %d\n", N_B_row,N_B_col);
    printf("Result Matrix C size: %d x %d\n", N_A_row,N_B_col);
    printf("Execution time(seconds): %.6f\n", max_time);
    for(int i=0; i<N_A_row; i++){                 //iterate over rows of C
      double rowSum=0.0;
      for(int j=0; j<N_B_col; j++){               //iterate over columns of C
        rowSum += C[(long)i*N_B_col + j];
      }
      if (N_A_row <= 50 && N_B_col <= 50) { // Only print row sums if result matrix dimensions < 50
        printf(" Row %2d sum = %.2f\n", i,rowSum);
      }
      checksum+=rowSum;
    }
    printf("Checksum (sum of all elements)= %.6e\n", checksum);
  }

  // Free memory
  free(B);      //allocated on all processes
  free(A_rows);
  free(C_rows);
  if(rank==0){
    free(A);
    free(C);
    free(recvcounts);
    free(displs);
  }

  MPI_Finalize();
  return 0;
}

In [ ]:
!mpicc -O3 mpi_mat_mul.c -o mpi_mat_mul

In [ ]:
!mpirun --allow-run-as-root --oversubscribe -np 4 ./mpi_mat_mul 4 3 3 5


In [ ]:
!mpirun --allow-run-as-root --oversubscribe -np 2 ./mpi_mat_mul 40 4000 4000 40

In [ ]:
!mpirun --allow-run-as-root --oversubscribe -np 4 ./mpi_mat_mul 40 4000 4000 40

In [ ]:
!mpirun --allow-run-as-root --oversubscribe -np 1 ./mpi_mat_mul 4000 400 400 4000

In [ ]:
!mpirun --allow-run-as-root --oversubscribe -np 2 ./mpi_mat_mul 4000 400 400 4000

In [ ]:
!mpirun --allow-run-as-root --oversubscribe -np 4 ./mpi_mat_mul 4000 400 400 4000

### **CUDA Code**

In [ ]:
%%writefile cuda_mat_mul.cu
// Compile: nvcc -O3 -arch=sm_75 cuda_mat_mul.cu -o cuda_mat_mul
// Run: ./cuda_mat_mul N_A_row N_A_col N_B_row N_B_col blockSize

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>      //for runtime API functions

//macro for CUDA error checking
#define CHECK_CUDA_ERROR(val) check((val), #val, __FILE__, __LINE__)
void check(cudaError_t err, const char* const func, const char* const file, const int line){
  if (err!=cudaSuccess){
    fprintf (stderr, "CUDA Error at %s:%d in function %s: %s\n", file, line, func, cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

float* alloc_mat(int rows, int cols){
  float *m = (float*) malloc((size_t)rows*cols*sizeof(float));
  if (!m){
    perror("malloc");
    exit(1);
  }
  return m;
}

void init_ex(float *A, float *B, int N_A_row, int N_A_col, int N_B_row, int N_B_col){
  //initialize Matrix A
  for (long i=0; i<(long)N_A_row*N_A_col; i++){
    A[i] = (float)((i/N_A_col)+(i%N_A_col)+1);
  }
  //initialize Matrix B
  for (long i=0; i<(long)N_B_row*N_B_col; i++){
    B[i] = (float)( ((i/N_B_col)+1) * ((i%N_B_col)+2));
  }
}

void print_mat(const char *name, float *M, int rows, int cols){
  printf("%s:\n", name);
  for (int i=0; i<rows; i++){
    for (int j=0; j<cols; j++){
      printf("%8.2f ", M[(long)i*cols+j]);
    }
    printf("\n");
  }
}

//CUDA kernel for matrix multiplication
__global__ void matmul_kernel(float *A, float *B, float *C, int N_A_row, int N_A_col, int N_B_row, int N_B_col){
  int row = blockIdx.y * blockDim.y + threadIdx.y;
  int col = blockIdx.x * blockDim.x + threadIdx.x;

  if (row < N_A_row && col < N_B_col){
    float sum = 0.0f;
    for (int k=0; k<N_A_col; k++){
      sum += A[(long)row*N_A_col+k] * B[(long)k*N_B_col+col];
    }
    C[(long)row*N_B_col + col]=sum;
  }
}

int main(int argc, char **argv){
  if (argc<6){
    printf ("Usage: %s N_A_row N_A_col N_B_row N_B_col blockSize\n", argv[0]);
    return 1;
  }

  int N_A_row = atoi(argv[1]);
  int N_A_col = atoi(argv[2]);
  int N_B_row = atoi(argv[3]);
  int N_B_col = atoi(argv[4]);
  int blockSize = atoi(argv[5]);

  if (N_A_row<=0 || N_A_col<=0 || N_B_row<=0 || N_B_col<=0 || blockSize<=0){
    printf("Invalid arguments. All dimensions and block size must be positive.\n");
    return 1;
  }

  if (N_A_col != N_B_row){
    printf("Error: N_A_col must be equal to N_B_row for matrix multiplication.\n");
    return 1;
  }

  //host matrices allocation
  float *A = alloc_mat(N_A_row, N_A_col);
  float *B = alloc_mat(N_B_row, N_B_col);
  float *C = alloc_mat(N_A_row, N_B_col);   //Result matrix C will have dimensions N_A_row x N_B_col

  init_ex(A,B,N_A_row, N_A_col, N_B_row, N_B_col);

  //initialize host C to zeros(to debugging comparison in kernel failures)
  for (long i=0; i<(long)N_A_row*N_B_col; i++){
    C[i] = 0.0f;
  }

  //device memory allocation
  float *d_A, *d_B, *d_C;
  CHECK_CUDA_ERROR(cudaMalloc((void**)&d_A, (size_t)N_A_row*N_A_col*sizeof(float)));
  CHECK_CUDA_ERROR(cudaMalloc((void**)&d_B, (size_t)N_B_row*N_B_col*sizeof(float)));
  CHECK_CUDA_ERROR(cudaMalloc((void**)&d_C, (size_t)N_A_row*N_B_col*sizeof(float)));

  // Copy data -> device
  CHECK_CUDA_ERROR(cudaMemcpy(d_A, A, (size_t)N_A_row*N_A_col*sizeof(float), cudaMemcpyHostToDevice));
  CHECK_CUDA_ERROR(cudaMemcpy(d_B, B, (size_t)N_B_row*N_B_col*sizeof(float), cudaMemcpyHostToDevice));
  //didn't copy host C to device <-- d_C is written to device by kernel

  //configure grid and block
  dim3 dimBlock(blockSize, blockSize);
  dim3 dimGrid( (N_B_col+blockSize-1)/blockSize, (N_A_row+blockSize-1)/blockSize);   //grid dimensions based on C (N_A_row x N_B_col)

  //CUDA events for timing
  cudaEvent_t start, stop;
  CHECK_CUDA_ERROR(cudaEventCreate(&start));
  CHECK_CUDA_ERROR(cudaEventCreate(&stop));
  CHECK_CUDA_ERROR(cudaEventRecord(start));

  // Launch kernel
  matmul_kernel <<<dimGrid, dimBlock>>> (d_A, d_B, d_C, N_A_row, N_A_col, N_B_row, N_B_col);
  CHECK_CUDA_ERROR(cudaGetLastError());   //check for errors from kernel launch

  //wait for kernel to finish
  CHECK_CUDA_ERROR(cudaDeviceSynchronize());

  CHECK_CUDA_ERROR(cudaEventRecord(stop));
  CHECK_CUDA_ERROR(cudaEventSynchronize(stop));

  float milliseconds = 0;
  CHECK_CUDA_ERROR(cudaEventElapsedTime (&milliseconds, start, stop));
  float seconds = milliseconds/1000.0f; //convert milliseconds to seconds

  //copy result back to host
  CHECK_CUDA_ERROR(cudaMemcpy(C, d_C, (size_t)N_A_row*N_B_col*sizeof(float), cudaMemcpyDeviceToHost));

  //print matrices(when N_A_row and N_B_col are smaller than 10)
  if (N_A_row<=10 && N_A_col<=10 && N_B_row<=10 && N_B_col<=10){
    print_mat("Matrix A", A,N_A_row, N_A_col);
    printf("\n");
    print_mat("Matrix B", B,N_B_row, N_B_col);
    printf("\n");
    print_mat("Result Matrix C = A * B", C,N_A_row, N_B_col);
    printf("\n");
  }

  //compute checksum and row sums
  float checksum = 0.0f;
  printf("Summary:\n");
  printf("Matrix A size: %d x %d \n", N_A_row, N_A_col);
  printf("Matrix B size: %d x %d \n", N_B_row, N_B_col);
  printf("Result Matrix C size: %d x %d \n", N_A_row, N_B_col);
  printf("Execution time(seconds): %.6f\n", seconds);
  for(int i=0; i<N_A_row; i++){                //iterate over rows of C
    float rowSum = 0.0f;
    for (int j=0; j<N_B_col; j++){             //iterate over columns of C
      rowSum += C[(long)i*N_B_col + j];
    }
    if (N_A_row <= 50 && N_B_col <= 50) {   // Only print row sums if result matrix dimensions < 50
      printf(" Row %2d sum = %.2f\n", i,rowSum);
    }
    checksum += rowSum;
  }
  printf("Checksum (sum of all elements) = %.6e\n", checksum);

  //free memory
  free(A);
  free(B);
  free(C);
  CHECK_CUDA_ERROR(cudaFree(d_A));
  CHECK_CUDA_ERROR(cudaFree(d_B));
  CHECK_CUDA_ERROR(cudaFree(d_C));
  CHECK_CUDA_ERROR(cudaEventDestroy(start));
  CHECK_CUDA_ERROR(cudaEventDestroy(stop));

  return 0;
}

In [ ]:
!nvcc -O3 -arch=sm_75 cuda_mat_mul.cu -o cuda_mat_mul

In [ ]:
!./cuda_mat_mul 4 3 3 5 4

In [ ]:
!./cuda_mat_mul 40 4000 4000 40 4

In [ ]:
!./cuda_mat_mul 40 4000 4000 40 8

In [ ]:
!./cuda_mat_mul 4000 400 400 4000 1

In [ ]:
!./cuda_mat_mul 4000 400 400 4000 2

In [ ]:
!./cuda_mat_mul 4000 400 400 4000 4

In [ ]:
!./cuda_mat_mul 4000 400 400 4000 8